# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 17 2022 2:22PM,237428,20,SSNA-2173615866,Sartorius Stedim North America,Released
1,May 17 2022 2:17PM,237427,19,GP52022,"Granules Pharmaceuticals, Inc.",Released
2,May 17 2022 2:17PM,237426,20,ALU0006838,Alumier Labs Inc.,Released
3,May 17 2022 2:17PM,237426,20,ALUR091043502,Alumier Labs Inc.,Released
4,May 17 2022 2:17PM,237426,20,ALUR228850215,Alumier Labs Inc.,Released
5,May 17 2022 2:17PM,237426,20,ALUR368124019,Alumier Labs Inc.,Released
6,May 17 2022 2:17PM,237426,20,ALUR593792559,Alumier Labs Inc.,Released
7,May 17 2022 2:17PM,237426,20,ALUR624578397,Alumier Labs Inc.,Released
8,May 17 2022 2:17PM,237426,20,ALUR732581719,Alumier Labs Inc.,Released
9,May 17 2022 2:17PM,237426,20,ALUR841718881,Alumier Labs Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
41,237421,Released,1
42,237425,Released,1
43,237426,Released,20
44,237427,Released,1
45,237428,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
237421,NaN,NaN,1.0
237425,NaN,NaN,1.0
237426,NaN,NaN,20.0
237427,NaN,NaN,1.0
237428,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237023,0.0,1.0,0.0
237323,0.0,1.0,0.0
237326,0.0,0.0,1.0
237327,2.0,1.0,0.0
237336,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237023,0,1,0
237323,0,1,0
237326,0,0,1
237327,2,1,0
237336,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237023,0,1,0
1,237323,0,1,0
2,237326,0,0,1
3,237327,2,1,0
4,237336,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237023,,1,
1,237323,,1,
2,237326,,,1
3,237327,2,1,
4,237336,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 17 2022 2:22PM,237428,20,Sartorius Stedim North America
1,May 17 2022 2:17PM,237427,19,"Granules Pharmaceuticals, Inc."
2,May 17 2022 2:17PM,237426,20,Alumier Labs Inc.
22,May 17 2022 1:53PM,237409,10,"Citieffe, Inc."
27,May 17 2022 1:44PM,237425,20,Sartorius Stedim North America
28,May 17 2022 1:38PM,237421,20,Sartorius Stedim North America
29,May 17 2022 1:31PM,237420,20,Sartorius Stedim North America
30,May 17 2022 1:30PM,237419,19,"AdvaGen Pharma, Ltd"
35,May 17 2022 1:17PM,237417,19,ACG North America LLC
36,May 17 2022 1:14PM,237416,19,ACG North America LLC


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,May 17 2022 2:22PM,237428,20,Sartorius Stedim North America,,,1
1,May 17 2022 2:17PM,237427,19,"Granules Pharmaceuticals, Inc.",,,1
2,May 17 2022 2:17PM,237426,20,Alumier Labs Inc.,,,20
3,May 17 2022 1:53PM,237409,10,"Citieffe, Inc.",,,5
4,May 17 2022 1:44PM,237425,20,Sartorius Stedim North America,,,1
5,May 17 2022 1:38PM,237421,20,Sartorius Stedim North America,,,1
6,May 17 2022 1:31PM,237420,20,Sartorius Stedim North America,,,1
7,May 17 2022 1:30PM,237419,19,"AdvaGen Pharma, Ltd",,,5
8,May 17 2022 1:17PM,237417,19,ACG North America LLC,,,1
9,May 17 2022 1:14PM,237416,19,ACG North America LLC,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 17 2022 2:22PM,237428,20,Sartorius Stedim North America,1,,
1,May 17 2022 2:17PM,237427,19,"Granules Pharmaceuticals, Inc.",1,,
2,May 17 2022 2:17PM,237426,20,Alumier Labs Inc.,20,,
3,May 17 2022 1:53PM,237409,10,"Citieffe, Inc.",5,,
4,May 17 2022 1:44PM,237425,20,Sartorius Stedim North America,1,,
5,May 17 2022 1:38PM,237421,20,Sartorius Stedim North America,1,,
6,May 17 2022 1:31PM,237420,20,Sartorius Stedim North America,1,,
7,May 17 2022 1:30PM,237419,19,"AdvaGen Pharma, Ltd",5,,
8,May 17 2022 1:17PM,237417,19,ACG North America LLC,1,,
9,May 17 2022 1:14PM,237416,19,ACG North America LLC,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 17 2022 2:22PM,237428,20,Sartorius Stedim North America,1,,
1,May 17 2022 2:17PM,237427,19,"Granules Pharmaceuticals, Inc.",1,,
2,May 17 2022 2:17PM,237426,20,Alumier Labs Inc.,20,,
3,May 17 2022 1:53PM,237409,10,"Citieffe, Inc.",5,,
4,May 17 2022 1:44PM,237425,20,Sartorius Stedim North America,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 17 2022 2:22PM,237428,20,Sartorius Stedim North America,1.0,NaN,NaN
1,May 17 2022 2:17PM,237427,19,"Granules Pharmaceuticals, Inc.",1.0,NaN,NaN
2,May 17 2022 2:17PM,237426,20,Alumier Labs Inc.,20.0,NaN,NaN
3,May 17 2022 1:53PM,237409,10,"Citieffe, Inc.",5.0,NaN,NaN
4,May 17 2022 1:44PM,237425,20,Sartorius Stedim North America,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 17 2022 2:22PM,237428,20,Sartorius Stedim North America,1.0,0.0,0.0
1,May 17 2022 2:17PM,237427,19,"Granules Pharmaceuticals, Inc.",1.0,0.0,0.0
2,May 17 2022 2:17PM,237426,20,Alumier Labs Inc.,20.0,0.0,0.0
3,May 17 2022 1:53PM,237409,10,"Citieffe, Inc.",5.0,0.0,0.0
4,May 17 2022 1:44PM,237425,20,Sartorius Stedim North America,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2611338,87.0,56.0,2.0
12,237342,0.0,1.0,0.0
15,237363,6.0,4.0,5.0
16,1186820,4.0,1.0,0.0
18,474346,0.0,2.0,0.0
19,949679,8.0,0.0,0.0
20,3086171,71.0,0.0,0.0
21,237381,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2611338,87.0,56.0,2.0
1,12,237342,0.0,1.0,0.0
2,15,237363,6.0,4.0,5.0
3,16,1186820,4.0,1.0,0.0
4,18,474346,0.0,2.0,0.0
5,19,949679,8.0,0.0,0.0
6,20,3086171,71.0,0.0,0.0
7,21,237381,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,87.0,56.0,2.0
1,12,0.0,1.0,0.0
2,15,6.0,4.0,5.0
3,16,4.0,1.0,0.0
4,18,0.0,2.0,0.0
5,19,8.0,0.0,0.0
6,20,71.0,0.0,0.0
7,21,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,87.0
1,12,Released,0.0
2,15,Released,6.0
3,16,Released,4.0
4,18,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,20,21
Status,,,,,,,,
Completed,2.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
Executing,56.0,1.0,4.0,1.0,2.0,0.0,0.0,1.0
Released,87.0,0.0,6.0,4.0,0.0,8.0,71.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,20,21
0,Completed,2.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
1,Executing,56.0,1.0,4.0,1.0,2.0,0.0,0.0,1.0
2,Released,87.0,0.0,6.0,4.0,0.0,8.0,71.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,20,21
0,Completed,2.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
1,Executing,56.0,1.0,4.0,1.0,2.0,0.0,0.0,1.0
2,Released,87.0,0.0,6.0,4.0,0.0,8.0,71.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()